In [1]:
'''
author: zhilin Zhang
'''
data_file = './data/data.conll'
dict_file = './data/word.dict'
sentences_file = './data/data.txt'
cut_file = './data/data.out'

In [2]:
def read_data():
    sentences = []
    words = []
    cut_sentences = []
    with open(data_file, 'r', encoding='utf-8') as f:
        line = f.readline()
        sentence = ''
        cut_sentence = []
        while line:
            if line == '\n':
                sentences.append(sentence)
                cut_sentences.append(' '.join(cut_sentence))
                cut_sentence = []
                sentence = ''
                line = f.readline()
            else:
                line = line.replace('\n', '')
                word = line.split('\t')[1]
                words.append(word)
                cut_sentence.append(word)
                sentence = sentence + word
                line = f.readline()
    words = set(words)
    return sentences, cut_sentences, words

In [3]:
def save_data(iter_obj, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for i in iter_obj:
            f.write(i)
            f.write('\n')

In [4]:
# sentences: sequntional stencences
# cut_sentences: standard cut result of sentences
#words: words dict(set)
sentences, cut_sentences, words = read_data()

In [5]:
def forward_max_match(text, words_dict):
    p1 = 0
    p2 = len(text)
    words = []
    while p1 < p2:
        sub_text = text[p1: p2]
        if p2 - p1 == 1:
            words.append(sub_text)
            p1 = p1 + 1
            p2 = len(text)
        elif sub_text in words_dict:
            words.append(sub_text)
            p1 = p2
            p2 = len(text)
        else:
            p2 = p2 - 1
    return words

In [6]:
# save file 'data.out'
with open(cut_file, 'w', encoding='utf-8') as f:
    for i, stc in enumerate(sentences):
        cut_result = forward_max_match(stc, words)
        f.write(' '.join(cut_result))
        f.write('\n')

In [24]:
# compute precision, recall and f1 value
precision = 0
recall = 0
for i, text in enumerate(sentences):
    target = cut_sentences[i].split()
    cut = forward_max_match(text, words)
    target_ = [[p, p+len(t)-1] for p,t in enumerate(target)]
    cut_ = [[p, p+len(t)-1] for p,t in enumerate(cut)]
    itsc = []
    for t in cut_:
        if t in target_:
            itsc.append(t)
    p = len(itsc) / len(cut)
    r = len(itsc) / len(target)
    precision = precision + p
    recall = recall + r
precision = precision / i
recall = recall / i
f1 = (2 * precision * recall) / (precision + recall)
print('%s:%f, %s:%f, %s:%f' % ('precision', precision, 'recall', recall, 'f1', f1))

precision:0.977186, recall:0.975585, f1:0.976385
